<a href="https://colab.research.google.com/github/SeonHyungJo/tensorflow-2.0-study/blob/master/TensorFlow_Tutorial/Keras_Tuner_%EC%86%8C%EA%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요
Keras Tuner는 TensorFlow 프로그램을위한 최적의 하이퍼 파라미터 세트를 선택하는 데 도움이되는 라이브러리입니다. 당신의 기계 학습 (ML) 응용 프로그램에 대한 하이퍼 파라미터의 오른쪽 집합을 선택하는 과정은 hyperparameter 조정 또는 hypertuning이라고합니다.

하이퍼 파라미터는 학습 프로세스와 ML 모델의 토폴로지를 제어하는 ​​변수입니다. 이러한 변수는 훈련 프로세스 동안 일정하게 유지되며 ML 프로그램의 성능에 직접적인 영향을 미칩니다. 초 매개 변수에는 두 가지 유형이 있습니다.

- 히든 레이어의 수와 너비와 같은 모델 선택에 영향을 미치는 모델 하이퍼 파라미터
- SGD (Stochastic Gradient Descent)의 학습률 및 KNN (Nearest Neighbors) 분류기의 최근 접 이웃 수와 같은 학습 알고리즘의 속도와 품질에 영향을 미치는 알고리즘 하이퍼 파라미터

이 자습서에서는 Keras Tuner를 사용하여 이미지 분류 응용 프로그램에 대한 하이퍼 튜닝을 수행합니다.


In [ ]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 61kB 1.8MB/s 


# 데이터 세트 다운로드 및 준비
이 자습서에서는 Keras Tuner를 사용하여 Fashion MNIST 데이터 세트 에서 의류 이미지를 분류하는 기계 학습 모델에 가장 적합한 하이퍼 파라미터를 찾습니다.

데이터를로드합니다

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# 모델 정의
하이퍼 튜닝을위한 모델을 빌드 할 때 모델 아키텍처 외에 하이퍼 파라미터 검색 공간도 정의합니다. 하이퍼 튜닝을 위해 설정 한 모델을 하이퍼 모델이라고합니다.

두 가지 접근 방식을 통해 하이퍼 모델을 정의 할 수 있습니다.

- 모델 빌더 기능을 사용하여
- 서브 클래 싱으로 HyperModel Keras 튜너 API의 클래스를
또한 두 개의 미리 정의 사용할 수 있습니다 HyperModel - 클래스 HyperXception을 하고 HyperResNet을 컴퓨터 비전 애플리케이션을 위해.

이 자습서에서는 모델 작성기 함수를 사용하여 이미지 분류 모델을 정의합니다. 모델 작성기 함수는 컴파일 된 모델을 반환하고 인라인으로 정의한 하이퍼 파라미터를 사용하여 모델을 하이퍼 튜닝합니다.

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

# 튜너를 인스턴스화하고 하이퍼 튜닝을 수행합니다.
튜너를 인스턴스화하여 하이퍼 튜닝을 수행합니다. Hyperband Tuner에는 RandomSearch , Hyperband , BayesianOptimization 및 Sklearn 네 가지 튜너가 있습니다. 이 자습서에서는 하이퍼 밴드 튜너를 사용합니다.

하이퍼 밴드 튜너를 인스턴스화하려면 하이퍼 모델, 최적화 할 objective 및 훈련 할 최대 max_epochs ( max_epochs )를 지정해야합니다.

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')                       

하이퍼 밴드 튜닝 알고리즘은 적응 형 리소스 할당 및 조기 중지를 사용하여 고성능 모델에 빠르게 수렴합니다. 이것은 스포츠 챔피언십 스타일 브래킷을 사용하여 수행됩니다. 이 알고리즘은 몇 epoch 동안 많은 수의 모델을 훈련하고 최고 성능의 모델 절반 만 다음 라운드로 전달합니다. 하이퍼 밴드는 1 + 로그 factor ( max_epochs )를 계산하고 가장 가까운 정수로 반올림하여 대괄호에서 훈련 할 모델 수를 결정합니다.

하이퍼 파라미터 검색을 실행하기 전에 모든 학습 단계가 끝날 때 학습 출력을 지우는 콜백을 정의하십시오.


In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

초 매개 변수 검색을 실행합니다. 검색 메서드의 인수는 위의 콜백 외에도 tf.keras.model.fit 에 사용 된 인수와 동일합니다.

In [ ]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Epoch 1/2
1875/1875 [==============================] - ETA: 0s - loss: 0.5916 - accuracy: 0.7858

KeyboardInterrupt: ignored

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

NameError: ignored

my_dir/intro_to_kt 디렉토리에는 하이퍼 파라미터 검색 중에 실행 된 모든 시험 (모델 구성)에 대한 자세한 로그와 체크 포인트가 포함되어 있습니다. 초 매개 변수 검색을 다시 실행하면 Keras Tuner는 이러한 로그의 기존 상태를 사용하여 검색을 재개합니다. 이 동작을 비활성화하려면 튜너를 인스턴스화하는 동안 추가 overwrite = True 인수를 전달하십시오.
